In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
from multiprocessing import reduction
import warnings
from collections import OrderedDict
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import Compose, Normalize, ToTensor
from tqdm import tqdm
from sklearn.metrics import r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from numpy import vstack
from sklearn.preprocessing import StandardScaler

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
data_path= '/content/drive/My Drive/INTERN_2022/DRUG/data/sensitivity_data/new_aug2/' 

In [7]:
GDSC=pd.read_csv(data_path+'GDSC_paired_common_with_gcsi.csv', index_col=0)
gCSI=pd.read_csv(data_path+'gCSI_paired_common_with_gdsc.csv', index_col=0)

In [8]:
GDSC_x = GDSC.iloc[:, 0:-1]
GDSC_y = GDSC.iloc[:, -1]

In [9]:
gCSI = gCSI.reset_index()
gCSI = gCSI.drop(columns='index')

In [10]:
gCSI_x = gCSI.iloc[:, 0:-1]
gCSI_y = gCSI.iloc[:, -1]

In [11]:
gCSI_x.iloc[0:2, 0:-20]

,155,864,53,681,50,707,667,102,548,26,...,1193,1194,1195,1196,1197,1198,1199,1200,1201,1202
0,7.209434,5.274817,6.520254,5.407016,5.441268,6.193294,4.710726,7.727200,4.280433,5.037682,...,3.776118,3.251995,-0.23223,-2.542907,-1.106178,-1.244095,-2.526046,-4.232081,-0.197347,0
1,7.694390,4.846773,5.213439,4.534354,6.048548,5.649415,5.006172,7.226347,9.675669,4.332932,...,3.776118,3.251995,-0.23223,-2.542907,-1.106178,-1.244095,-2.526046,-4.232081,-0.197347,0


In [12]:
gCSI_x.iloc[:, 0:-21] = StandardScaler().fit_transform(gCSI_x.iloc[:, 0:-21])
GDSC_x.iloc[:, 0:-21] = StandardScaler().fit_transform(GDSC_x.iloc[:, 0:-21])


In [13]:
from sklearn.model_selection import train_test_split

gCSI_X_train, gCSI_X_test, gCSI_y_train, gCSI_y_test = train_test_split(gCSI_x, gCSI_y, test_size = 0.30, random_state = 42) 
GDSC_X_train, GDSC_X_test, GDSC_y_train, GDSC_y_test = train_test_split(GDSC_x, GDSC_y,test_size = 0.30, random_state = 42)

In [14]:
gCSI_train = pd.concat([gCSI_X_train, gCSI_y_train], axis = 1)

In [15]:
gCSI_test = pd.concat([gCSI_X_test, gCSI_y_test], axis = 1)

In [16]:
GDSC_test = pd.concat([GDSC_X_test, GDSC_y_test], axis = 1)

In [17]:
GDSC_train = pd.concat([GDSC_X_train, GDSC_y_train], axis = 1)

In [18]:
len(GDSC_test.columns)

422

custom dataset for dataloader

In [19]:
seed = 0
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [20]:


class GDSC_Dataset(Dataset):
    # load the dataset
    def __init__(self, path, train_val_test = 1):
        # load the csv file as a dataframe
        df = path#read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, 0:len(df.columns)-1]
        self.y = df.values[:, len(df.columns)-1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        self.X = StandardScaler().fit_transform(self.X)
        # label encode target and ensure the values are floats
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
        x_train, x_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.3, random_state=42)
        #x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, train_size=0.5, random_state=0, stratify = y_test)

        self.train_val_test = train_val_test
        # two modes - train and test
        if( train_val_test ==1):
            self.x_data, self.y_data = x_train, y_train
        else:
            self.x_data, self.y_data = x_test, y_test

    # number of rows in the dataset
    def __len__(self):
        return len(self.x_data)

    # get a row at an index
    def __getitem__(self, idx):
        #print(self.x_data[idx].shape, self.y_data[idx].shape)
        return [self.x_data[idx], self.y_data[idx]]



In [21]:


class mixed_Dataset_for_test(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the csv file as a dataframe
        df = path#read_csv(path, header=None)
        # store the inputs and outputs
        self.X = df.values[:, 0:len(df.columns)-1]
        self.y = df.values[:, len(df.columns)-1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        self.X = StandardScaler().fit_transform(self.X)
        # label encode target and ensure the values are floats
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))

    # number of rows in the dataset
    def __len__(self):
        return len(self.X)

    # get a row at an index
    def __getitem__(self, idx):
        #print(self.x_data[idx].shape, self.y_data[idx].shape)
        return [self.X[idx], self.y[idx]]



In [22]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor

# Create linear regression object
# change to RandomForestRegressor() while needed 
#m1 = RandomForestRegressor(max_depth=4)#linear_model.Ridge()
#m2=  RandomForestRegressor(max_depth=4)#linear_model.Ridge()

In [62]:
class Net(nn.Module):
    """Model (simple CNN adapted from 'PyTorch: A 60 Minute Blitz')"""

    def __init__(self) -> None:
        super(Net, self).__init__()
        #self.lin = nn.Linear(620,1)
        self.layers = nn.Sequential(
        nn.Linear(len(GDSC_train.columns)-1, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.layers(x)
        return x

In [24]:
def train(net, trainloader, epochs):
    """Train the model on the training set."""
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    # enumerate epochs
    for epoch in range(epochs):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(trainloader):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = net(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

In [25]:
def test(net, testloader):
    """Validate the model on the test set."""
    net.eval()
    criterion = torch.nn.MSELoss()
    correct, total, loss = 0, 0, 0.0
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(testloader):
        # evaluate the model on the test set
        yhat = net(inputs)
        # retrieve numpy array

        #yhat = yhat.detach().numpy()
        #actual = targets.numpy()
        actual = targets.reshape((len(targets), 1))

        loss += criterion(yhat,actual).item()

        #print(yhat)
        predictions.append(yhat.detach().numpy())
        actuals.append(actual.detach().numpy())
    
    #print(type(predictions))
    predictions, actuals= vstack(predictions), vstack(actuals)

    
    print("Mean squared error: %.2f" % mean_squared_error(actuals,predictions))

    #mean absolute error
    print("Mean absolute error: %.2f" % mean_absolute_error(actuals, predictions))

    # The coefficient of determination: 1 is perfect prediction
    print("Coefficient of determination: %.2f" % r2_score(actuals, predictions))

    return mean_squared_error(actuals,predictions), mean_absolute_error(actuals, predictions), r2_score(actuals, predictions)

In [26]:
warnings.filterwarnings("ignore", category=UserWarning)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [27]:
def load_data(path):
    # load the dataset
    train = GDSC_Dataset(path,train_val_test=1)
    test =GDSC_Dataset(path,train_val_test=3)
    batch_size =16

    # prepare data loaders
    train_dl = DataLoader(train, batch_size=batch_size, shuffle=True)
    test_dl = DataLoader(test, batch_size=batch_size, shuffle=False)
    return train_dl, test_dl

In [28]:
m1 = Net().to(DEVICE)
m2= Net().to(DEVICE)

In [29]:
GDSC_trainloader, GDSC_testloader = load_data(GDSC)
gCSI_trainloader, gCSI_testloader = load_data(gCSI)

In [30]:
# make a class prediction for one row of data
def predict(net, testloader):
    net.eval()
    criterion = torch.nn.MSELoss()
    correct, total, loss = 0, 0, 0.0
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(testloader):
        # evaluate the model on the test set
        yhat = net(inputs)
        # retrieve numpy array

        #yhat = yhat.detach().numpy()
        #actual = targets.numpy()
        actual = targets.reshape((len(targets), 1))

        loss += criterion(yhat,actual).item()

        #print(yhat)
        predictions.append(yhat.detach().numpy())
        actuals.append(actual.detach().numpy())
    
    #print(type(predictions))
    predictions, actuals= vstack(predictions), vstack(actuals)
    return predictions, actuals


In [32]:
train(m1, GDSC_trainloader, epochs=100)
train(m2, gCSI_trainloader, epochs = 100)

In [33]:
#in-study test results GDSC 30%
mse_loss_GDSC, mae_loss_GDSC, r2_GDSC = test(m1, GDSC_testloader)
print(mse_loss_GDSC, mae_loss_GDSC, r2_GDSC)

Mean squared error: 0.01
Mean absolute error: 0.07
Coefficient of determination: 0.59
0.01044592 0.07453356 0.5852139513931452


In [34]:
#in-study test results GDSC 30%
mse_loss_gCSI, mae_loss_gCSI, r2_gCSI = test(m2, gCSI_testloader)
print(mse_loss_gCSI, mae_loss_gCSI, r2_gCSI)

Mean squared error: 0.01
Mean absolute error: 0.07
Coefficient of determination: 0.61
0.0083500855 0.06802776 0.6058813244696439


In [35]:
#predict(GDSC_X_test.iloc[0,:], m1)

In [36]:
#mixed study results trained on GDSC (70%)

mixed_test = pd.concat([GDSC_test, gCSI_test], ignore_index=True)

mixed_test = mixed_Dataset_for_test(mixed_test)


In [37]:
mixed_test_dl = DataLoader(mixed_test, batch_size=16, shuffle=False)

In [38]:
#test m1 on mixed test data
mse_loss_mixed_m1, mae_loss_mixed_m1, r2_mixed_m1 = test(m1, mixed_test_dl)
print(mse_loss_mixed_m1, mae_loss_mixed_m1, r2_mixed_m1)

Mean squared error: 0.02
Mean absolute error: 0.10
Coefficient of determination: 0.19
0.0188206 0.09553614 0.18829512698449402


In [39]:
#test m2 on mixed test data
mse_loss_mixed_m2, mae_loss_mixed_m2, r2_mixed_m2 = test(m2, mixed_test_dl)
print(mse_loss_mixed_m2, mae_loss_mixed_m2, r2_mixed_m2)

Mean squared error: 0.02
Mean absolute error: 0.10
Coefficient of determination: 0.12
0.02040544 0.09804788 0.11994333272996094


In [40]:
#m5

mixed_train = pd.concat([GDSC_train, gCSI_train], ignore_index=True)

mixed_train = mixed_Dataset_for_test(mixed_train)

mixed_train_dl = DataLoader(mixed_train, batch_size=16, shuffle=True)

m5 = Net().to(DEVICE)

In [41]:
train(m5, mixed_train_dl, epochs=100)
#train(m2, gCSI_trainloader, epochs = 100)

In [42]:
#test m5 on mixed test data
mse_loss_mixed_m5, mae_loss_mixed_m5, r2_mixed_m5 = test(m5, mixed_test_dl)
print(mse_loss_mixed_m5, mae_loss_mixed_m5, r2_mixed_m5)

Mean squared error: 0.01
Mean absolute error: 0.09
Coefficient of determination: 0.41
0.013625166 0.08542629 0.4123666341314963


In [43]:
GDSC_y_train_pred =test(m1, GDSC_trainloader)
gCSI_y_train_pred = test(m2, gCSI_trainloader)

Mean squared error: 0.00
Mean absolute error: 0.05
Coefficient of determination: 0.83
Mean squared error: 0.00
Mean absolute error: 0.05
Coefficient of determination: 0.84


In [44]:
GDSC_y_train_pred, GDSC_y_train_m1 = predict(m1, GDSC_trainloader)
gCSI_y_train_pred, gCSI_y_train_m2 = predict(m2, gCSI_trainloader)

In [45]:
GDSC_y_train_pred

array([[0.29964817],
       [0.52472603],
       [0.26454476],
       ...,
       [0.06338625],
       [0.09169172],
       [0.24674943]], dtype=float32)

WA method

In [46]:
GDSC_y_pred, GDSC_y_test_m1 = predict(m1, GDSC_testloader)
gCSI_y_pred, gCSI_y_test_m2 = predict(m2, gCSI_testloader)

In [47]:
avg_y_pred = (GDSC_y_pred + gCSI_y_pred)/2

In [48]:
avg_y_pred = pd.Series(avg_y_pred.squeeze())

In [49]:
avg_y_pred_stacked = pd.concat([avg_y_pred, avg_y_pred], ignore_index=True)

In [50]:
mixed_y_test1 = pd.concat([pd.Series(GDSC_y_pred.squeeze()),pd.Series(gCSI_y_pred.squeeze())], ignore_index =True)

In [51]:
print("Mean squared error: %.2f" % mean_squared_error(mixed_y_test1, avg_y_pred_stacked))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(mixed_y_test1, avg_y_pred_stacked))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(mixed_y_test1, avg_y_pred_stacked))

Mean squared error: 0.01
Mean absolute error: 0.05
Coefficient of determination: 0.70


do a linear regression on the training scores

In [52]:
m4 = linear_model.LinearRegression()

In [53]:
m4.fit(gCSI_y_train_pred, GDSC_y_train_pred)

# Make predictions using the testing set
GDSC_y_pred_m4 = m4.predict(gCSI_y_pred.reshape(-1,1))

#avg GDSC values predicted by m4 + gCSI input y values  to m4 
gCSI = (GDSC_y_pred_m4 + gCSI_y_pred.reshape(-1,1))/2

#print(gCSI)
print("Mean squared error: %.2f" % mean_squared_error(gCSI_y_test, gCSI))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(gCSI_y_test, gCSI))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(gCSI_y_test, gCSI))

Mean squared error: 0.01
Mean absolute error: 0.08
Coefficient of determination: 0.50


In [54]:
m3 =linear_model.LinearRegression()

In [55]:
m3.fit(GDSC_y_train_pred, gCSI_y_train_pred)

# Make predictions using the testing set
gCSI_y_pred_m3 = m3.predict(GDSC_y_pred.reshape(-1,1))

GDSC = (gCSI_y_pred_m3 + GDSC_y_pred.reshape(-1,1))/2
print("Mean squared error: %.2f" % mean_squared_error(GDSC_y_test, GDSC))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(GDSC_y_test, GDSC))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(GDSC_y_test, GDSC))

Mean squared error: 0.01
Mean absolute error: 0.09
Coefficient of determination: 0.46


In [56]:
GDSC = pd.Series(GDSC.squeeze())
gCSI =pd.Series(gCSI.squeeze())

In [57]:
concated = pd.concat([GDSC, gCSI], ignore_index=True)

In [58]:
#test on mixed data 

print("Mean squared error: %.2f" % mean_squared_error(mixed_y_test1, concated))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(mixed_y_test1, concated))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(mixed_y_test1, concated))

Mean squared error: 0.00
Mean absolute error: 0.05
Coefficient of determination: 0.73


FL

In [59]:
params1 = []
for param in m1.parameters():
  params1.append(param.data)

params2 = []
for param in m2.parameters():
  params2.append(param.data)

In [63]:
m6 = Net().to(DEVICE)
params3 = iter(params1 + params2)
for param in m6.parameters():
  param.data = next(params3)/2

In [64]:
for param in m6.parameters():
  print(param.data)

tensor([[-0.0050,  0.0108, -0.0243,  ...,  0.0072,  0.0180, -0.0016],
        [-0.0117,  0.0042,  0.0116,  ...,  0.0048, -0.0036,  0.0172],
        [-0.0021,  0.0035,  0.0040,  ..., -0.0169, -0.0074,  0.0183],
        ...,
        [ 0.0106,  0.0073,  0.0022,  ...,  0.0019,  0.0096,  0.0206],
        [ 0.0209,  0.0186, -0.0212,  ...,  0.0252,  0.0195, -0.0045],
        [-0.0180, -0.0043, -0.0173,  ..., -0.0175, -0.0055, -0.0073]])
tensor([ 4.3594e-03,  1.8363e-02, -2.0327e-02,  1.9914e-02,  9.6901e-03,
         2.0135e-02, -1.7248e-03, -1.5142e-02, -1.8233e-02,  1.8629e-02,
        -1.7780e-02, -1.6315e-02, -7.8250e-03, -2.4336e-02, -7.8545e-04,
        -1.4614e-03,  4.2537e-03,  2.3740e-04, -1.4404e-02,  1.9076e-02,
        -1.0033e-02, -2.6687e-02,  2.0731e-02, -1.8220e-02, -1.9025e-02,
        -2.4066e-02,  1.3998e-02,  7.2796e-03,  1.8053e-02,  1.1360e-02,
        -1.5066e-02,  8.2651e-03,  3.0104e-03,  1.6438e-02, -1.3620e-02,
         2.1556e-02, -6.2377e-03, -1.5507e-02, -6.3817e

In [65]:
#test m6 on mixed test data
mse_loss_mixed_m6, mae_loss_mixed_m6, r2_mixed_m6 = test(m6, mixed_test_dl)
print(mse_loss_mixed_m6, mae_loss_mixed_m6, r2_mixed_m6)

Mean squared error: 0.04
Mean absolute error: 0.14
Coefficient of determination: -0.65
0.038311947 0.13526098 -0.6523380390331499


In [66]:
m6_test_pred, mixed_m6_gtruth = predict(m6, mixed_test_dl)

In [67]:
print("Mean squared error: %.2f" % mean_squared_error(mixed_m6_gtruth, m6_test_pred))

#mean absolute error
print("Mean absolute error: %.2f" % mean_absolute_error(mixed_m6_gtruth, m6_test_pred))

# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(mixed_m6_gtruth, m6_test_pred))

Mean squared error: 0.04
Mean absolute error: 0.14
Coefficient of determination: -0.65
